<a href="https://colab.research.google.com/github/iject/SB3/blob/main/optuna/optuna_lab_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter tuning with Optuna

Github repo: https://github.com/araffin/tools-for-robotic-rl-icra2022

Optuna: https://github.com/optuna/optuna

Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

Documentation: https://stable-baselines3.readthedocs.io/en/master/

SB3 Contrib: https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) is a collection of pre-trained Reinforcement Learning agents using Stable-Baselines3.

It also provides basic scripts for training, evaluating agents, tuning hyperparameters and recording videos.


## Introduction

In this notebook, you will learn the importance of tuning hyperparameters. You will first try to optimize the parameters manually and then we will see how to automate the search using Optuna.


## Install Dependencies and Stable Baselines3 Using Pip

List of full dependencies can be found in the [README](https://github.com/DLR-RM/stable-baselines3).


```
pip install stable-baselines3[extra]
```

In [27]:
!pip install stable-baselines3
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 14.7 MB/s eta 0:00:00


In [2]:
# Optional: install SB3 contrib to have access to additional algorithms
!pip install sb3-contrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 6.0 MB/s eta 0:00:00


In [3]:
# Optuna will be used in the last part when doing hyperparameter tuning
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.6 MB/s eta 0:00:00


## Imports

In [4]:
import gymnasium as gym
import numpy as np

The first thing you need to import is the RL model, check the documentation to know what you can use on which problem

In [5]:
from stable_baselines3 import PPO, A2C, SAC, TD3, DQN

In [6]:
# Algorithms from the contrib repo
# https://github.com/Stable-Baselines-Team/stable-baselines3-contrib
from sb3_contrib import QRDQN, TQC

In [7]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

# Part I: The Importance Of Tuned Hyperparameters



When compared with Supervised Learning, Deep Reinforcement Learning is far more sensitive to the choice of hyper-parameters such as learning rate, number of neurons, number of layers, optimizer ... etc.

Poor choice of hyper-parameters can lead to poor/unstable convergence. This challenge is compounded by the variability in performance across random seeds (used to initialize the network weights and the environment).

In addition to hyperparameters, selecting the appropriate algorithm is also an important choice. We will demonstrate it on the simple Pendulum task.

See [gym doc](https://gym.openai.com/envs/Pendulum-v0/): "The inverted pendulum swingup problem is a classic problem in the control literature. In this version  of the problem, the pendulum starts in a random position, and the goal is to swing it up so it stays upright."


Let's try first with PPO and a small budget of 4000 steps (20 episodes):

In [8]:
env_id = "Pendulum-v1"
# Env used only for evaluation
eval_envs = make_vec_env(env_id, n_envs=10)
# 4000 training timesteps
budget_pendulum = 4000

### PPO

In [ ]:
ppo_model = PPO("MlpPolicy", env_id, seed=0, verbose=0).learn(budget_pendulum)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

PPO Mean episode reward: -1177.32 +/- 255.97


### A2C

In [ ]:
# Define and train a A2C model
a2c_model = A2C("MlpPolicy", env_id, seed=0, verbose=0).learn(budget_pendulum)

In [ ]:
# Evaluate the train A2C model
mean_reward, std_reward = evaluate_policy(a2c_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"A2C Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

A2C Mean episode reward: -1324.29 +/- 318.48


Both are far from solving the env (mean reward around -200).
Now, let's try with an off-policy algorithm:

### Training longer PPO ?

Maybe training longer would help?

You can try with 10x the budget, but in the case of A2C/PPO, training longer won't help much, finding better hyperparameters is needed instead.

In [ ]:
# train longer
new_budget = 10 * budget_pendulum

ppo_model = PPO("MlpPolicy", env_id, seed=0, verbose=0).learn(new_budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

PPO Mean episode reward: -1226.19 +/- 280.93


### PPO - Tuned Hyperparameters

Using Optuna, we can in fact tune the hyperparameters and find a working solution (from the [RL Zoo](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/ppo.yml)):

In [ ]:
tuned_params = {
    "gamma": 0.9,
    "use_sde": True,
    "sde_sample_freq": 4,
    "learning_rate": 1e-3,
}

# budget = 10 * budget_pendulum
ppo_tuned_model = PPO("MlpPolicy", env_id, seed=1, verbose=1, **tuned_params).learn(50_000, log_interval=5)

Using cpu device
Creating environment from the given name 'Pendulum-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -1.2e+03    |
| time/                   |             |
|    fps                  | 661         |
|    iterations           | 5           |
|    time_elapsed         | 15          |
|    total_timesteps      | 10240       |
| train/                  |             |
|    approx_kl            | 0.033493523 |
|    clip_fraction        | 0.157       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.62       |
|    explained_variance   | 0.811       |
|    learning_rate        | 0.001       |
|    loss                 | 14          |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0132     |
|    std                  | 0.933       |
|    value_

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_tuned_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"Tuned PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Tuned PPO Mean episode reward: -158.56 +/- 100.66


Note: if you try SAC on the simple MountainCarContinuous environment, you will encounter some issues without tuned hyperparameters: https://github.com/rail-berkeley/softlearning/issues/76

Simple environments can be challenging even for SOTA algorithms.

# Part II: Grad Student Descent


### Challenge (10 minutes): "Grad Student Descent"
The challenge is to find the best hyperparameters (max performance) for A2C on `CartPole-v1` with a limited budget of 20 000 training steps.


Maximum reward: 500 on `CartPole-v1`

The hyperparameters should work for different random seeds.

In [9]:
budget = 20_000

#### The baseline: default hyperparameters

In [10]:
eval_envs_cartpole = make_vec_env("CartPole-v1", n_envs=10)

In [11]:
model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=0).learn(budget)

Using cpu device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 25.4     |
|    ep_rew_mean        | 25.4     |
| time/                 |          |
|    fps                | 351      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.518   |
|    explained_variance | 0.571    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 2.16     |
|    value_loss         | 7.77     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 26.4     |
|    ep_rew_mean        | 26.4     |
| time/                 |          |
|    fps                | 411      |


In [12]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:126.12 +/- 12.20


**Your goal is to beat that baseline and get closer to the optimal score of 500**

Time to tune!

In [13]:
import torch.nn as nn

In [14]:
policy_kwargs = dict(
    net_arch=[
      dict(vf=[64, 64], pi=[64, 64]), # network architectures for actor/critic
    ],
    activation_fn=nn.Tanh,
)

hyperparams = dict(
    n_steps=5, # number of steps to collect data before updating policy
    learning_rate=7e-4,
    gamma=0.99, # discount factor
    max_grad_norm=0.5, # The maximum value for the gradient clipping
    ent_coef=0.0, # Entropy coefficient for the loss calculation
)

model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=0, **hyperparams).learn(budget)

In [15]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:128.70 +/- 13.47


Hint - Recommended Hyperparameter Range

```python
gamma = trial.suggest_float("gamma", 0.9, 0.99999, log=True)
max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
# from 2**3 = 8 to 2**10 = 1024
n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
# net_arch tiny: {"pi": [64], "vf": [64]}
# net_arch default: {"pi": [64, 64], "vf": [64, 64]}
# activation_fn = nn.Tanh / nn.ReLU
```

# Part III: Automatic Hyperparameter Tuning





In this part we will create a script that allows to search for the best hyperparameters automatically.

### Imports

In [16]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

### Config

In [17]:
N_TRIALS = 100  # Maximum number of trials
N_JOBS = 1 # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 2  # Number of evaluations during the training
N_TIMESTEPS = int(2e4)  # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 15)  # 15 minutes

ENV_ID = "CartPole-v1"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": ENV_ID,
}

### Exercise (5 minutes): Define the search space

In [20]:
from typing import Any, Dict, Optional
import torch
import torch.nn as nn

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for A2C hyperparameters.

    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """
    # Discount factor between 0.9 and 0.9999
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    # 8, 16, 32, ... 1024
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)

    ### YOUR CODE HERE
    # TODO:
    # - define the learning rate search space [1e-5, 1] (log) -> `suggest_float`
    # - define the network architecture search space ["tiny", "small"] -> `suggest_categorical`
    # - define the activation function search space ["tanh", "relu"]
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    ### END OF YOUR CODE

    # Display true values
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]}
        if net_arch == "tiny"
        else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
        },
    }

### Define the objective function

First we define a custom callback to report the results of periodic evaluations to Optuna:

In [23]:
import os

In [25]:
log_path_EC = r"log_path/EvalCallback"
os.makedirs(log_path_EC, exist_ok=True)

In [26]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
        log_path: Optional[str] = None,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
            log_path=log_path,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### Exercise (10 minutes): Define the objective function

Then we define the objective function that is in charge of sampling hyperparameters, creating the model and then returning the result to Optuna

In [27]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    ### YOUR CODE HERE
    # TODO:
    # 1. Sample hyperparameters and update the default keyword arguments: `kwargs.update(other_params)`
    # 2. Create the evaluation envs
    # 3. Create the `TrialEvalCallback`

    # 1. Sample hyperparameters and update the keyword arguments
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model
    model = A2C(**kwargs)

    # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
    eval_env = make_vec_env(ENV_ID, N_EVAL_ENVS)

    # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
    # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
    # TrialEvalCallback signature:
    # TrialEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)
    eval_callback = TrialEvalCallback(eval_env, trial,
                                      N_EVAL_EPISODES, EVAL_FREQ,
                                      verbose=0, log_path=log_path_EC)

    ### END OF YOUR CODE

    nan_encountered = False
    try:
        # Train the model
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_envs.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

### The optimization loop

In [28]:
%%time
# ~15 minuts

import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

[I 2025-05-13 11:39:38,138] A new study created in memory with name: no-name-2b90683f-4db3-4fdf-a0ae-ecc199ce7d87
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Eval num_timesteps=10000, episode_reward=9.40 +/- 0.66
Episode length: 9.40 +/- 0.66
New best mean reward!


[I 2025-05-13 11:40:05,108] Trial 0 finished with value: 9.5 and parameters: {'gamma': 0.0012048481146359712, 'max_grad_norm': 4.707126915205051, 'exponent_n_steps': 6, 'lr': 0.007165665181062419, 'net_arch': 'small', 'activation_fn': 'tanh'}. Best is trial 0 with value: 9.5.


Eval num_timesteps=20000, episode_reward=9.50 +/- 0.81
Episode length: 9.50 +/- 0.81
New best mean reward!
Eval num_timesteps=10000, episode_reward=485.40 +/- 29.25
Episode length: 485.40 +/- 29.25
New best mean reward!


[I 2025-05-13 11:40:25,481] Trial 1 finished with value: 500.0 and parameters: {'gamma': 0.012815196432391299, 'max_grad_norm': 1.7474648131234445, 'exponent_n_steps': 6, 'lr': 0.006068960116412594, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 1 with value: 500.0.


...

Eval num_timesteps=10000, episode_reward=343.70 +/- 129.20
Episode length: 343.70 +/- 129.20
New best mean reward!
Eval num_timesteps=10000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
New best mean reward!


[I 2025-05-13 11:54:34,233] Trial 58 finished with value: 9.8 and parameters: {'gamma': 0.010568363052910559, 'max_grad_norm': 1.2299475998513585, 'exponent_n_steps': 5, 'lr': 0.008023760008201592, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 1 with value: 500.0.


Eval num_timesteps=20000, episode_reward=9.80 +/- 0.60
Episode length: 9.80 +/- 0.60


[I 2025-05-13 11:54:45,036] Trial 59 pruned. 


Eval num_timesteps=10000, episode_reward=194.60 +/- 23.95
Episode length: 194.60 +/- 23.95
New best mean reward!
Number of finished trials:  60
Best trial:
  Value: 500.0
  Params: 
    gamma: 0.012815196432391299
    max_grad_norm: 1.7474648131234445
    exponent_n_steps: 6
    lr: 0.006068960116412594
    net_arch: tiny
    activation_fn: tanh
  User attrs:
    gamma_: 0.9871848035676087
    n_steps: 64


Complete example: https://github.com/DLR-RM/rl-baselines3-zoo

In [32]:
study.best_params

{'gamma': 0.012815196432391299,
 'max_grad_norm': 1.7474648131234445,
 'exponent_n_steps': 6,
 'lr': 0.006068960116412594,
 'net_arch': 'tiny',
 'activation_fn': 'tanh'}

In [47]:
  display(study.best_trial)
  display(len(study.trials))


FrozenTrial(number=1, state=1, values=[500.0], datetime_start=datetime.datetime(2025, 5, 13, 11, 40, 5, 113075), datetime_complete=datetime.datetime(2025, 5, 13, 11, 40, 25, 481307), params={'gamma': 0.012815196432391299, 'max_grad_norm': 1.7474648131234445, 'exponent_n_steps': 6, 'lr': 0.006068960116412594, 'net_arch': 'tiny', 'activation_fn': 'tanh'}, user_attrs={'gamma_': 0.9871848035676087, 'n_steps': 64}, system_attrs={}, intermediate_values={1: 485.4, 2: 500.0}, distributions={'gamma': FloatDistribution(high=0.1, log=True, low=0.0001, step=None), 'max_grad_norm': FloatDistribution(high=5.0, log=True, low=0.3, step=None), 'exponent_n_steps': IntDistribution(high=10, log=False, low=3, step=1), 'lr': FloatDistribution(high=1.0, log=True, low=1e-05, step=None), 'net_arch': CategoricalDistribution(choices=('tiny', 'small')), 'activation_fn': CategoricalDistribution(choices=('tanh', 'relu'))}, trial_id=1, value=None)

60

In [37]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [38]:
plot_optimization_history(study)


In [39]:
plot_intermediate_values(study)


In [40]:
plot_parallel_coordinate(study)


In [42]:
optuna.importance.get_param_importances(study)

{'lr': np.float64(0.5276217753400368),
 'gamma': np.float64(0.18475264828839788),
 'max_grad_norm': np.float64(0.1256818169856007),
 'activation_fn': np.float64(0.12460765292023145),
 'exponent_n_steps': np.float64(0.037247955233250446),
 'net_arch': np.float64(8.815123248268194e-05)}

In [43]:
optuna.visualization.plot_param_importances(study)

# Conclusion

What we have seen in this notebook:
- the importance of good hyperparameters
- how to do automatic hyperparameter search with optuna


---

# Далее мои разработки

# Part IV: Automatic Hyperparameter Tuning для vec_env





In this part we will create a script that allows to search for the best hyperparameters automatically.

In [46]:
data = np.load(r"/content/log_path/EvalCallback/evaluations.npz")
print(data)
lst = data.files
print(lst)
for item in lst:
    print(item)
    print(data[item])


NpzFile '/content/log_path/EvalCallback/evaluations.npz' with keys: timesteps, results, ep_lengths
['timesteps', 'results', 'ep_lengths']
timesteps
[10000]
results
[[174. 177. 179. 212. 220. 169. 169. 213. 192. 241.]]
ep_lengths
[[174 177 179 212 220 169 169 213 192 241]]


### Imports

In [ ]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

### Config

In [48]:
N_TRIALS = 100  # Maximum number of trials
N_JOBS = 1 # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 2  # Number of evaluations during the training
N_TIMESTEPS = int(2e4)  # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 15)  # 15 minutes

# для train_env
N_ENVS = N_EVAL_ENVS
EVAL_FREQ = max(EVAL_FREQ // N_ENVS, 1)

ENV_ID = "CartPole-v1"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
}

### Exercise (5 minutes): Define the search space

In [49]:
from typing import Any, Dict, Optional
import torch
import torch.nn as nn

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for A2C hyperparameters.

    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """
    # Discount factor between 0.9 and 0.9999
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    # 8, 16, 32, ... 1024
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)

    ### YOUR CODE HERE
    # TODO:
    # - define the learning rate search space [1e-5, 1] (log) -> `suggest_float`
    # - define the network architecture search space ["tiny", "small"] -> `suggest_categorical`
    # - define the activation function search space ["tanh", "relu"]
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    ### END OF YOUR CODE

    # Display true values
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]}
        if net_arch == "tiny"
        else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
        },
    }

### Define the objective function

First we define a custom callback to report the results of periodic evaluations to Optuna:

In [50]:
import os

In [51]:
log_path_EC = r"log_path/EvalCallback"
os.makedirs(log_path_EC, exist_ok=True)

In [52]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
        log_path: Optional[str] = None,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
            log_path=log_path,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### Exercise (10 minutes): Define the objective function

Then we define the objective function that is in charge of sampling hyperparameters, creating the model and then returning the result to Optuna

In [53]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    ### YOUR CODE HERE
    # TODO:
    # 1. Sample hyperparameters and update the default keyword arguments: `kwargs.update(other_params)`
    # 2. Create the evaluation envs
    # 3. Create the `TrialEvalCallback`

    # 1. Sample hyperparameters and update the keyword arguments
    kwargs.update(sample_a2c_params(trial))
    # Create the RL model
    train_env = make_vec_env(ENV_ID, N_ENVS)
    model = A2C(env=train_env, **kwargs)

    # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
    eval_env = make_vec_env(ENV_ID, N_EVAL_ENVS)

    # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
    # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
    # TrialEvalCallback signature:
    # TrialEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)
    eval_callback = TrialEvalCallback(eval_env, trial,
                                      N_EVAL_EPISODES, EVAL_FREQ,
                                      verbose=0, log_path=log_path_EC)

    ### END OF YOUR CODE

    nan_encountered = False
    try:
        # Train the model
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_envs.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

### The optimization loop

In [54]:
import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS, multivariate=True)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

/usr/local/lib/python3.11/dist-packages/optuna/_experimental.py:31: ExperimentalWarning:

Argument ``multivariate`` is an experimental feature. The interface can change in the future.

[I 2025-05-13 12:48:41,823] A new study created in memory with name: no-name-ae91113b-e754-4d45-a61a-83251135f7a1
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/policies.py:486: UserWarning:

As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])



Eval num_timesteps=10000, episode_reward=283.20 +/- 130.66
Episode length: 283.20 +/- 130.66
New best mean reward!


[I 2025-05-13 12:48:48,303] Trial 0 finished with value: 500.0 and parameters: {'gamma': 0.002022566695992499, 'max_grad_norm': 0.9534889257609054, 'exponent_n_steps': 5, 'lr': 0.004627032684513558, 'net_arch': 'tiny', 'activation_fn': 'relu'}. Best is trial 0 with value: 500.0.


Eval num_timesteps=20000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
New best mean reward!
Eval num_timesteps=10000, episode_reward=70.10 +/- 17.48
Episode length: 70.10 +/- 17.48
New best mean reward!


[I 2025-05-13 12:48:55,851] Trial 1 finished with value: 88.7 and parameters: {'gamma': 0.024523345068315085, 'max_grad_norm': 0.3980820626059061, 'exponent_n_steps': 7, 'lr': 2.171859073238636e-05, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 0 with value: 500.0.


...

Eval num_timesteps=10000, episode_reward=9.20 +/- 0.60
Episode length: 9.20 +/- 0.60
New best mean reward!
Eval num_timesteps=10000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
New best mean reward!


[I 2025-05-13 12:54:21,508] Trial 82 finished with value: 500.0 and parameters: {'gamma': 0.0005932690904182689, 'max_grad_norm': 0.5706951929750083, 'exponent_n_steps': 5, 'lr': 0.0143123319817666, 'net_arch': 'tiny', 'activation_fn': 'relu'}. Best is trial 0 with value: 500.0.


Eval num_timesteps=20000, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00


[W 2025-05-13 12:54:22,926] Trial 83 failed with parameters: {'gamma': 0.0009902212289901145, 'max_grad_norm': 0.8793536051091422, 'exponent_n_steps': 5, 'lr': 0.004964937803482774, 'net_arch': 'tiny', 'activation_fn': 'relu'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-53-008036110673>", line 42, in objective
    model.learn(N_TIMESTEPS, callback=eval_callback)
  File "/usr/local/lib/python3.11/dist-packages/stable_baselines3/a2c/a2c.py", line 201, in learn
    return super().learn(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py", line 324, in learn
    continue_training = self.collect_rollouts(self.env, callback, self.rollout_buffer, n_rollout_steps=self.n_steps)
                

Number of finished trials:  84
Best trial:
  Value: 500.0
  Params: 
    gamma: 0.002022566695992499
    max_grad_norm: 0.9534889257609054
    exponent_n_steps: 5
    lr: 0.004627032684513558
    net_arch: tiny
    activation_fn: relu
  User attrs:
    gamma_: 0.9979774333040075
    n_steps: 32


In [ ]:
Params:
    gamma: 0.002022566695992499
    max_grad_norm: 0.9534889257609054
    exponent_n_steps: 5
    lr: 0.004627032684513558
    net_arch: tiny
    activation_fn: relu
  User attrs:
    gamma_: 0.9979774333040075
    n_steps: 32

  hyperparams = {}

# Part V: Теперь для Ant-v5

In this part we will create a script that allows to search for the best hyperparameters automatically.

### Imports

In [8]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

### Config

In [19]:
N_TRIALS = 100  # Maximum number of trials
N_JOBS = 1 # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 2  # Number of evaluations during the training
# N_WARMUP_STEPS = N_EVALUATIONS // 3 # Do not prune before 1/3 of the max budget is used
N_WARMUP_STEPS = N_STARTUP_TRIALS + 3 # Do not prune before 1/3 of the max budget is used
N_TIMESTEPS = int(1e3)  # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 15)  # 15 minutes

# для train_env
N_ENVS = 1
EVAL_FREQ = max(EVAL_FREQ // N_ENVS, 1)

ENV_ID = "Ant-v5"
ALGO = SAC

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
}

### Exercise (5 minutes): Define the search space

In [21]:
from typing import Any, Dict, Optional
import torch
import torch.nn as nn

def sample_SAC_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for SAC hyperparameters.

    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """
    one_minus_gamma = trial.suggest_float("one_minus_gamma", 0.0001, 0.03, log=True)
    # From 2**5=32 to 2**11=2048
    batch_size_pow = trial.suggest_int("batch_size_pow", 2, 11)

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 0.002, log=True)
    train_freq = trial.suggest_int("train_freq", 1, 10)
    gradient_steps = trial.suggest_int("gradient_steps", 1, 10)
    # Polyak coeff
    tau = trial.suggest_float("tau", 0.001, 0.08, log=True)

    # NOTE: Add "verybig" to net_arch when tuning HER
    net_arch = trial.suggest_categorical("net_arch", ["small", "medium", "big"])
    # activation_fn = trial.suggest_categorical('activation_fn', [nn.Tanh, nn.ReLU, nn.ELU, nn.LeakyReLU])

    trial.set_user_attr("gamma", 1 - one_minus_gamma)
    trial.set_user_attr("batch_size", 2**batch_size_pow)

    net_arch = {
        "small": [64, 64],
        "medium": [256, 256],
        "big": [400, 300],
        # "large": [256, 256, 256],
        # "verybig": [512, 512, 512],
    }[net_arch]

    hyperparams = {
        "gamma": 1 - one_minus_gamma,
        "batch_size": 2**batch_size_pow,
        "learning_rate": learning_rate,
        "train_freq": train_freq,
        "gradient_steps": gradient_steps,
        "tau": tau,
        "policy_kwargs": {
            "net_arch": net_arch,
            # "activation_fn": activation_fn,
        },
    }



    return hyperparams

### Define the objective function

First we define a custom callback to report the results of periodic evaluations to Optuna:

In [22]:
import os

In [23]:
log_path_EC = r"log_path/EvalCallback"
os.makedirs(log_path_EC, exist_ok=True)

In [24]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
        log_path: Optional[str] = None,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
            log_path=log_path,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### Exercise (10 minutes): Define the objective function

Then we define the objective function that is in charge of sampling hyperparameters, creating the model and then returning the result to Optuna

In [29]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    ### YOUR CODE HERE
    # TODO:
    # 1. Sample hyperparameters and update the default keyword arguments: `kwargs.update(other_params)`
    # 2. Create the evaluation envs
    # 3. Create the `TrialEvalCallback`

    # 1. Sample hyperparameters and update the keyword arguments
    kwargs.update(sample_SAC_params(trial))
    # Create the RL model
    train_env = make_vec_env(ENV_ID, N_ENVS)
    model = ALGO(env=train_env, **kwargs)

    # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
    eval_env = make_vec_env(ENV_ID, N_EVAL_ENVS)

    # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
    # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
    # TrialEvalCallback signature:
    # TrialEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)
    eval_callback = TrialEvalCallback(eval_env, trial,
                                      N_EVAL_EPISODES, EVAL_FREQ,
                                      verbose=0, log_path=None)

    ### END OF YOUR CODE

    nan_encountered = False
    try:
        # Train the model
        print("Start train model")
        model.learn(N_TIMESTEPS, callback=eval_callback)
        print("End train model\n")
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

### The optimization loop

In [30]:
import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS, multivariate=True)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_WARMUP_STEPS
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_sac_ant-v5.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

/usr/local/lib/python3.11/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-05-14 04:39:28,677] A new study created in memory with name: no-name-ffb80845-2626-4210-aae1-3bc797eb3670


Start train model


[I 2025-05-14 04:40:23,260] Trial 0 finished with value: 914.4960729999999 and parameters: {'one_minus_gamma': 0.012477726234843946, 'batch_size_pow': 8, 'learning_rate': 0.001975504325557571, 'train_freq': 6, 'gradient_steps': 4, 'tau': 0.002330761938808824, 'net_arch': 'big'}. Best is trial 0 with value: 914.4960729999999.


End train model

Start train model


[I 2025-05-14 04:43:41,910] Trial 1 finished with value: -1441.8796455 and parameters: {'one_minus_gamma': 0.004391686162792903, 'batch_size_pow': 10, 'learning_rate': 0.00022409356822235054, 'train_freq': 4, 'gradient_steps': 5, 'tau': 0.05764443682687115, 'net_arch': 'big'}. Best is trial 0 with value: 914.4960729999999.


...

End train model

Start train model


[I 2025-05-14 04:53:07,068] Trial 17 finished with value: 1000.2393804 and parameters: {'one_minus_gamma': 0.00036488587408493964, 'batch_size_pow': 11, 'learning_rate': 3.94033779322373e-05, 'train_freq': 9, 'gradient_steps': 4, 'tau': 0.001014112412006081, 'net_arch': 'medium'}. Best is trial 13 with value: 1000.4074965000002.


End train model

Start train model


[I 2025-05-14 04:55:14,909] Trial 18 finished with value: 994.0387825 and parameters: {'one_minus_gamma': 0.00045394972509737086, 'batch_size_pow': 11, 'learning_rate': 1.7482017373797255e-05, 'train_freq': 8, 'gradient_steps': 5, 'tau': 0.0015692967931188676, 'net_arch': 'medium'}. Best is trial 13 with value: 1000.4074965000002.


End train model

Number of finished trials:  19
Best trial:
  Value: 1000.4074965000002
  Params: 
    one_minus_gamma: 0.001821742372963789
    batch_size_pow: 8
    learning_rate: 9.24939595501435e-05
    train_freq: 7
    gradient_steps: 1
    tau: 0.001242337252706291
    net_arch: medium
  User attrs:
    gamma: 0.9981782576270362
    batch_size: 256


In [37]:
display(study.best_params)
display(study.best_trial.params)
display(study.best_trial.user_attrs)

{'one_minus_gamma': 0.001821742372963789,
 'batch_size_pow': 8,
 'learning_rate': 9.24939595501435e-05,
 'train_freq': 7,
 'gradient_steps': 1,
 'tau': 0.001242337252706291,
 'net_arch': 'medium'}

{'one_minus_gamma': 0.001821742372963789,
 'batch_size_pow': 8,
 'learning_rate': 9.24939595501435e-05,
 'train_freq': 7,
 'gradient_steps': 1,
 'tau': 0.001242337252706291,
 'net_arch': 'medium'}

{'gamma': 0.9981782576270362, 'batch_size': 256}

In [39]:
best_trial = dict(**study.best_trial.params, **study.best_trial.user_attrs)
display(best_trial)
tuned_hyperparams = dict(gamma=best_trial["gamma"],
                         batch_size=best_trial["batch_size"],
                         learning_rate=best_trial["learning_rate"],
                         train_freq=best_trial["train_freq"],
                         gradient_steps=best_trial["gradient_steps"],
                         tau=best_trial["tau"],
                         policy_kwargs=dict(net_arch = [256, 256]),
                         )

{'one_minus_gamma': 0.001821742372963789,
 'batch_size_pow': 8,
 'learning_rate': 9.24939595501435e-05,
 'train_freq': 7,
 'gradient_steps': 1,
 'tau': 0.001242337252706291,
 'net_arch': 'medium',
 'gamma': 0.9981782576270362,
 'batch_size': 256}

In [46]:
vec_env = make_vec_env(ENV_ID, N_EVAL_ENVS-1)
tuned_model = ALGO("MlpPolicy", env=vec_env, **tuned_hyperparams).learn(300_000, progress_bar=True)

eval_env = make_vec_env(ENV_ID, N_EVAL_ENVS)

mean_reward, std_reward = evaluate_policy(tuned_model, eval_env)
print(f"{ALGO.__name__} Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

vec_env.close()
tuned_model.env.close()
eval_env.close()

Output()

SAC Mean episode reward: 479.08 +/- 29.34


In [47]:
print(f"study_results_{ALGO.__name__}_{ENV_ID}.csv")

study_results_SAC_Ant-v5.csv


In [48]:
import pickle

study_ant = None
with open('study_Ant-v5.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    study_ant = pickle.load(f)

In [49]:
plot_param_importances(study_ant)

In [50]:
plot_optimization_history(study_ant)

# Part VI: Теперь для Ant-v5 (VecEnv)

In this part we will create a script that allows to search for the best hyperparameters automatically.

### Imports

In [61]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

### Config

In [62]:
N_TRIALS = 100  # Maximum number of trials
N_JOBS = 1 # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 2  # Number of evaluations during the training
# N_WARMUP_STEPS = N_EVALUATIONS // 3 # Do not prune before 1/3 of the max budget is used
N_WARMUP_STEPS = N_STARTUP_TRIALS + 3 # Do not prune before 1/3 of the max budget is used
N_TIMESTEPS = int(0.1e6)  # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 60 * 5)  # 5 hours

# для train_env
N_ENVS = N_EVAL_ENVS
EVAL_FREQ = max(EVAL_FREQ // N_ENVS, 1)

ENV_ID = "Ant-v5"
ALGO = SAC

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    # "batch_size": 256,              # по-умолчанию
    # "tau": 0.02,                    # позже попробовать (default=0.005)
    # "buffer_size": 32768,           # позже попробовать (default=1000000)
    # "policy_kwargs": {
            # "net_arch": [400, 300], # (default=[256, 256])
            # "log_std_init": -4      # позже попробовать (default=-3)
        # },
}

### Exercise (5 minutes): Define the search space

In [63]:
from typing import Any, Dict, Optional
import torch
import torch.nn as nn

def sample_SAC_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for SAC hyperparameters.

    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """
    one_minus_gamma = trial.suggest_float("one_minus_gamma", 0.0001, 0.03, log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 0.002, log=True)
    train_freq = trial.suggest_int("train_freq", 1, 15)
    learning_starts = trial.suggest_int("learning_starts", 100, 50_000)
    gradient_steps = trial.suggest_int("gradient_steps", 1, 15)
    tau = trial.suggest_float("tau", 0.001, 0.08, log=True)

    trial.set_user_attr("gamma", 1 - one_minus_gamma)

    hyperparams = {
        "gamma": 1 - one_minus_gamma,
        "learning_rate": learning_rate,
        "train_freq": train_freq,
        "learning_starts": learning_starts,
        "gradient_steps": gradient_steps,
        "tau": tau,
    }

    return hyperparams

### Define the objective function

First we define a custom callback to report the results of periodic evaluations to Optuna:

In [64]:
import os
import time

In [65]:
log_path_EC = r"log_path/EvalCallback"
os.makedirs(log_path_EC, exist_ok=True)

In [66]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
        log_path: Optional[str] = None,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
            log_path=log_path,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### Exercise (10 minutes): Define the objective function

Then we define the objective function that is in charge of sampling hyperparameters, creating the model and then returning the result to Optuna

In [67]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    kwargs.update(sample_SAC_params(trial))

    train_env = make_vec_env(ENV_ID, N_ENVS)
    model = ALGO(env=train_env, **kwargs)

    eval_env = make_vec_env(ENV_ID, N_EVAL_ENVS)

    eval_callback = TrialEvalCallback(eval_env, trial,
                                      N_EVAL_EPISODES, EVAL_FREQ,
                                      verbose=0)

    nan_encountered = False
    try:
        print("Start train model")
        start_time = time.time()
        model.learn(N_TIMESTEPS, callback=eval_callback)
        print(f"End train model. Train time: {time.time() - start_time}\n")
    except AssertionError as e:
        print(e)
        nan_encountered = True
    finally:
        vec_env.close()
        model.env.close()
        eval_env.close()

    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

### The optimization loop

In [68]:
import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS, multivariate=True)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_WARMUP_STEPS
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv(f"study_results_{ALGO.__name__}_{ENV_ID}.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

/usr/local/lib/python3.11/dist-packages/optuna/_experimental.py:31: ExperimentalWarning:

Argument ``multivariate`` is an experimental feature. The interface can change in the future.

[I 2025-05-14 06:43:25,489] A new study created in memory with name: no-name-9e4c5605-62ad-4b43-ba4b-061f01b3ed5d


Start train model


[I 2025-05-14 07:03:00,114] Trial 0 finished with value: 777.8675183 and parameters: {'one_minus_gamma': 0.00016750007051566018, 'learning_rate': 0.0012840522387973963, 'train_freq': 1, 'learning_starts': 32603, 'gradient_steps': 2, 'tau': 0.004633077233065415}. Best is trial 0 with value: 777.8675183.


End train model. Train time: 1174.2450001239777

Start train model


[I 2025-05-14 07:09:14,081] Trial 1 finished with value: -2574.1259835000005 and parameters: {'one_minus_gamma': 0.0040461805868500155, 'learning_rate': 9.505465313707192e-05, 'train_freq': 8, 'learning_starts': 45988, 'gradient_steps': 5, 'tau': 0.07793480371287047}. Best is trial 0 with value: 777.8675183.


...

End train model. Train time: 2369.2687559127808

Start train model


[I 2025-05-14 09:11:43,191] Trial 7 finished with value: 492.8056063 and parameters: {'one_minus_gamma': 0.00025381282110940434, 'learning_rate': 0.001384028379181535, 'train_freq': 1, 'learning_starts': 30718, 'gradient_steps': 1, 'tau': 0.003075293569211277}. Best is trial 0 with value: 777.8675183.


End train model. Train time: 671.2271797657013

Start train model


[W 2025-05-14 09:14:09,919] Trial 8 failed with parameters: {'one_minus_gamma': 0.007558668778922603, 'learning_rate': 0.0014212258837454621, 'train_freq': 5, 'learning_starts': 25333, 'gradient_steps': 2, 'tau': 0.006187097413002806} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-67-bdb47f0c5964>", line 29, in objective
    model.learn(N_TIMESTEPS, callback=eval_callback)
  File "/usr/local/lib/python3.11/dist-packages/stable_baselines3/sac/sac.py", line 308, in learn
    return super().learn(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/off_policy_algorithm.py", line 328, in learn
    rollout = self.collect_rollouts(
              ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/st

Number of finished trials:  9
Best trial:
  Value: 777.8675183
  Params: 
    one_minus_gamma: 0.00016750007051566018
    learning_rate: 0.0012840522387973963
    train_freq: 1
    learning_starts: 32603
    gradient_steps: 2
    tau: 0.004633077233065415
  User attrs:
    gamma: 0.9998324999294843


In [69]:
import pickle

with open('study_Ant-v5(100k, colab).pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(study, f, pickle.HIGHEST_PROTOCOL)



---



# Optuna use example from Arrafin [post](https://araffin.github.io/post/optuna/)

In [ ]:
"""Optuna example that optimizes the hyperparameters of
a reinforcement learning agent using PPO implementation from Stable-Baselines3
on a Gymnasium environment.

This is a simplified version of what can be found in https://github.com/DLR-RM/rl-baselines3-zoo.

You can run this example as follows:
    $ python optimize_ppo.py

"""

from typing import Any

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
import torch
import torch.nn as nn


N_TRIALS = 500
N_STARTUP_TRIALS = 10
N_EVALUATIONS = 2
N_TIMESTEPS = 40_000
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 10

ENV_ID = "Pendulum-v1"
N_ENVS = 5

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
}


def sample_ppo_params(trial: optuna.Trial) -> dict[str, Any]:
    """Sampler for PPO hyperparameters."""
    # From 2**5=32 to 2**12=4096
    n_steps_pow = trial.suggest_int("n_steps_pow", 5, 12)
    gamma = trial.suggest_float("gamma", 0.97, 0.9999)
    learning_rate = trial.suggest_float("learning_rate", 3e-5, 3e-3, log=True)
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    n_steps = 2**n_steps_pow
    # Display true values
    trial.set_user_attr("n_steps", n_steps)
    # Convert to PyTorch objects
    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "policy_kwargs": {
            "activation_fn": activation_fn,
        },
    }


class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    vec_env = make_vec_env(ENV_ID, n_envs=N_ENVS)
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_ppo_params(trial))
    # Create the RL model.
    model = PPO(env=vec_env, **kwargs)
    # Create env used for evaluation.
    eval_env = make_vec_env(ENV_ID, n_envs=N_ENVS)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env,
        trial,
        n_eval_episodes=N_EVAL_EPISODES,
        eval_freq=max(EVAL_FREQ // N_ENVS, 1),
        deterministic=True,
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS, multivariate=True)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(
        n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
    )

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print(f"Number of finished trials: {len(study.trials)}")

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print(f"    {key}: {value}")

/usr/local/lib/python3.11/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-05-07 06:23:50,486] A new study created in memory with name: no-name-ec51315d-b57f-4e1d-8172-06178f875f08
[I 2025-05-07 06:24:22,000] Trial 0 finished with value: -1104.8031417 and parameters: {'n_steps_pow': 11, 'gamma': 0.9717752594505243, 'learning_rate': 0.0009183829962213949, 'activation_fn': 'tanh'}. Best is trial 0 with value: -1104.8031417.
[I 2025-05-07 06:24:52,831] Trial 1 finished with value: -1403.2242927 and parameters: {'n_steps_pow': 8, 'gamma': 0.9956854702881278, 'learning_rate': 0.00017373306505887276, 'activation_fn': 'relu'}. Best is trial 0 with value: -1104.8031417.
[I 2025-05-07 06:25:24,747] Trial 2 finished with value: -1461.2066082000003 and parameters: {'n_steps_pow': 6, 'gamma': 0.9735116678394656, 'learning_rate': 3.3852430611972214e-05, 'activation_fn

Number of finished trials: 19
Best trial:
  Value:  -163.77188689999997
  Params: 
    n_steps_pow: 8
    gamma: 0.9749712609115107
    learning_rate: 0.0018765871621028618
    activation_fn: relu
  User attrs:
    n_steps: 256
